In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

%matplotlib inline
pd.set_option('display.max_rows', 500)

# Group by apply

In [2]:
pd_JH_data=pd.read_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
pd_JH_data=pd_JH_data.sort_values('date',ascending=True).reset_index(drop=True).copy()
pd_JH_data.head()

,date,state,country,confirmed
0,2020-01-22,Alberta,Canada,0.0
1,2020-01-22,no,"Korea, South",1.0
2,2020-01-22,no,Kosovo,0.0
3,2020-01-22,no,Kuwait,0.0
4,2020-01-22,no,Kyrgyzstan,0.0


In [3]:
test_data=pd_JH_data[((pd_JH_data['country']=='Yemen')|
                      (pd_JH_data['country']=='Germany'))&
                     (pd_JH_data['date']>'2020-06-20')]


In [4]:
test_data.head()


,date,state,country,confirmed
40207,2020-06-21,no,Germany,191272.0
40258,2020-06-21,no,Yemen,941.0
40476,2020-06-22,no,Germany,191768.0
40527,2020-06-22,no,Yemen,967.0
40740,2020-06-23,no,Germany,192480.0


In [5]:
test_data.groupby(['country']).agg(np.max)

,date,state,confirmed
country,,,
Germany,2020-08-21,no,233029.0
Yemen,2020-08-21,no,1906.0


In [6]:
# %load ../src/features/build_features.py

import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)

def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate'''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    #assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope



In [7]:
test_data.groupby(['state','country']).agg(np.max)

date  confirmed
state country                      
no    Germany 2020-08-21   233029.0
      Yemen   2020-08-21     1906.0

In [8]:
#test_data.groupby(['state','country']).apply(get_doubling_time_via_regression)

In [9]:
def rolling_reg(df_input,col='confirmed'):
    
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)
    return result

In [10]:
test_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed')

state  country       
no     Germany  40207           NaN
                40476           NaN
                40740    317.615894
                41007    348.817770
                41273    433.013094
                41539    332.061803
                41805    356.862925
                42071    591.767631
                42337    666.886986
                42602    538.071724
                42869    459.344301
                43134    411.541317
                43400    442.723036
                43667    475.320451
                43933    530.732167
                44198    456.339492
                44465    482.869919
                44731    624.783202
                44997    603.893617
                45263    628.785677
                45529    563.128060
                45795    680.249858
                46061    848.985138
                46326    745.567970
                46592    564.813146
                46858    404.289738
                47125    348.851371
      

In [11]:
pd_DR_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed').reset_index()

In [12]:
pd_DR_result=pd_DR_result.rename(columns={'confirmed':'doubling_rate',
                             'level_2':'index'})
pd_DR_result.head()

,state,country,index,doubling_rate
0,Alberta,Canada,0,NaN
1,Alberta,Canada,465,NaN
2,Alberta,Canada,701,NaN
3,Alberta,Canada,998,NaN
4,Alberta,Canada,1231,NaN


In [13]:
pd_JH_data=pd_JH_data.reset_index()
pd_JH_data.head()

,index,date,state,country,confirmed
0,0,2020-01-22,Alberta,Canada,0.0
1,1,2020-01-22,no,"Korea, South",1.0
2,2,2020-01-22,no,Kosovo,0.0
3,3,2020-01-22,no,Kuwait,0.0
4,4,2020-01-22,no,Kyrgyzstan,0.0


In [14]:
pd_result_larg=pd.merge(pd_JH_data,pd_DR_result[['index','doubling_rate']],on=['index'],how='left')


In [15]:
pd_result_larg[pd_result_larg['country']=='Germany']

,index,date,state,country,confirmed,doubling_rate
43,43,2020-01-22,no,Germany,0.0,NaN
308,308,2020-01-23,no,Germany,0.0,NaN
575,575,2020-01-24,no,Germany,0.0,NaN
840,840,2020-01-25,no,Germany,0.0,NaN
1106,1106,2020-01-26,no,Germany,0.0,NaN
1373,1373,2020-01-27,no,Germany,1.0,0.666667
1639,1639,2020-01-28,no,Germany,4.0,0.833333
1904,1904,2020-01-29,no,Germany,4.0,2.000000
2171,2171,2020-01-30,no,Germany,4.0,inf
2437,2437,2020-01-31,no,Germany,5.0,8.666667


In [16]:
pd_result_larg=pd.merge(pd_JH_data,pd_DR_result[['index','doubling_rate']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,doubling_rate
0,0,2020-01-22,Alberta,Canada,0.0,NaN
1,1,2020-01-22,no,"Korea, South",1.0,NaN
2,2,2020-01-22,no,Kosovo,0.0,NaN
3,3,2020-01-22,no,Kuwait,0.0,NaN
4,4,2020-01-22,no,Kyrgyzstan,0.0,NaN


# Filtering the data with groupby apply 


In [17]:
from scipy import signal

def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function 
        it ensures that the data structure is kept'''
    window=5, 
    degree=1
    df_result=df_input
    
    filter_in=df_input[column].fillna(0) # attention with the neutral element here
    
    result=signal.savgol_filter(np.array(filter_in),
                           5, # window size used for filtering
                           1)
    df_result[column+'_filtered']=result
    return df_result

In [18]:
pd_filtered_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(savgol_filter).reset_index()

In [19]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_result[['index','confirmed_filtered']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,doubling_rate,confirmed_filtered
0,0,2020-01-22,Alberta,Canada,0.0,NaN,0.0
1,1,2020-01-22,no,"Korea, South",1.0,NaN,0.8
2,2,2020-01-22,no,Kosovo,0.0,NaN,0.0
3,3,2020-01-22,no,Kuwait,0.0,NaN,0.0
4,4,2020-01-22,no,Kyrgyzstan,0.0,NaN,0.0


# Filtered doubling rate


In [20]:

pd_filtered_doubling=pd_result_larg[['state','country','confirmed_filtered']].groupby(['state','country']).apply(rolling_reg,'confirmed_filtered').reset_index()

pd_filtered_doubling=pd_filtered_doubling.rename(columns={'confirmed_filtered':'doubling_rate_filtered',
                             'level_2':'index'})

pd_filtered_doubling.tail()

,state,country,index,doubling_rate_filtered
56653,no,Zimbabwe,55460,54.757039
56654,no,Zimbabwe,55658,46.956725
56655,no,Zimbabwe,55955,48.673197
56656,no,Zimbabwe,56190,44.892728
56657,no,Zimbabwe,56657,41.389573


In [21]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_doubling[['index','doubling_rate_filtered']],on=['index'],how='left')
pd_result_larg.tail()

,index,date,state,country,confirmed,doubling_rate,confirmed_filtered,doubling_rate_filtered
56653,56653,2020-08-21,no,Barbados,157.0,156.000000,157.2,119.923077
56654,56654,2020-08-21,no,Belarus,70111.0,451.316129,70089.0,529.575322
56655,56655,2020-08-21,no,Belgium,80894.0,113.333804,80796.6,133.638560
56656,56656,2020-08-21,no,Albania,8119.0,51.895765,8120.8,51.291050
56657,56657,2020-08-21,no,Zimbabwe,5815.0,66.678295,5854.0,41.389573


In [22]:
mask=pd_result_larg['confirmed']>100
pd_result_larg['doubling_rate_filtered_DR']=pd_result_larg['doubling_rate_filtered'].where(mask, other=np.NaN) 


In [23]:
pd_result_larg.head()

,index,date,state,country,confirmed,doubling_rate,confirmed_filtered,doubling_rate_filtered,doubling_rate_filtered_DR
0,0,2020-01-22,Alberta,Canada,0.0,NaN,0.0,NaN,NaN
1,1,2020-01-22,no,"Korea, South",1.0,NaN,0.8,NaN,NaN
2,2,2020-01-22,no,Kosovo,0.0,NaN,0.0,NaN,NaN
3,3,2020-01-22,no,Kuwait,0.0,NaN,0.0,NaN,NaN
4,4,2020-01-22,no,Kyrgyzstan,0.0,NaN,0.0,NaN,NaN


In [24]:
pd_result_larg.to_csv('../data/processed/COVID_final_set.csv',sep=';',index=False)